In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "0"

In [2]:
import torch

torch.cuda.empty_cache()

In [ ]:
import cv2
import boto3
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# AWS Rekognition 클라이언트 설정
rekognition_client = boto3.client('rekognition', region_name='ap-northeast-2')

# 얼굴 및 ID 정보를 저장할 딕셔너리 (임베딩과 고유 ID)
known_faces = {}  # {face_id: {'image': reference_image, 'name': 'Person X'}}

# ID 카운터 초기화
face_id_counter = 1

# 새로운 얼굴이 등장했을 때 ID를 부여하는 함수
def assign_new_face_id(reference_image):
    global face_id_counter
    face_id = face_id_counter
    known_faces[face_id] = {'image': reference_image, 'name': f'Person {face_id}'}
    face_id_counter += 1
    return face_id

# 얼굴 비교 함수 (AWS Rekognition)
def compare_face_with_known_faces(target_image):
    for face_id, face_data in known_faces.items():
        try:
            response = rekognition_client.compare_faces(
                SourceImage=face_data['image'],
                TargetImage=target_image,
                SimilarityThreshold=80  # 유사도 기준값 설정
            )
            if response['FaceMatches']:
                similarity = response['FaceMatches'][0]['Similarity']
                return face_id, similarity
        except rekognition_client.exceptions.InvalidParameterException as e:
            print(f"InvalidParameterException: {e}")
            continue
    return None, 0

# 웹캠 설정
cap = cv2.VideoCapture(2)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 웹캠에서 캡처된 프레임을 Rekognition에서 사용할 수 있도록 변환
    _, buffer = cv2.imencode('.jpg', frame)
    webcam_image = {"Bytes": buffer.tobytes()}

    # 기존 인물들과 비교하여 ID 할당
    face_id, similarity = compare_face_with_known_faces(webcam_image)

    if face_id:
        # 기존 인물일 경우
        print(f"Matched with ID: {face_id}, Similarity: {similarity:.2f}%")
        cv2.putText(frame, f'ID {face_id} (Similarity: {similarity:.2f}%)', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    else:
        # 새로운 인물일 경우 새로운 ID 할당
        face_id = assign_new_face_id(webcam_image)
        print(f"New face detected, assigned ID: {face_id}")
        cv2.putText(frame, f'New ID {face_id}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # 프레임 출력
    cv2.imshow('Re-Identification with AWS Rekognition', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 및 해제
cap.release()
cv2.destroyAllWindows()


### 기존 - 요청 주기 설정

In [ ]:
import cv2
import boto3
import time
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# AWS Rekognition 클라이언트 설정
rekognition_client = boto3.client('rekognition', region_name='ap-northeast-2')

# 얼굴 및 ID 정보를 저장할 딕셔너리 (임베딩과 고유 ID)
known_faces = {}  # {face_id: {'image': reference_image, 'name': 'Person X'}}
face_id_counter = 1  # ID 카운터 초기화

# 최근 Rekognition 요청 시간 기록
last_rekognition_request_time = 0
REQUEST_INTERVAL = 2  # 요청 간격 (초)

# 새로운 얼굴이 등장했을 때 ID를 부여하는 함수
def assign_new_face_id(reference_image):
    global face_id_counter
    face_id = face_id_counter
    known_faces[face_id] = {'image': reference_image, 'name': f'Person {face_id}'}
    face_id_counter += 1
    return face_id

# 얼굴 비교 함수 (AWS Rekognition)
def compare_face_with_known_faces(target_image):
    for face_id, face_data in known_faces.items():
        try:
            response = rekognition_client.compare_faces(
                SourceImage=face_data['image'],
                TargetImage=target_image,
                SimilarityThreshold=80  # 유사도 기준값 설정
            )
            if response['FaceMatches']:
                similarity = response['FaceMatches'][0]['Similarity']
                return face_id, similarity
        except rekognition_client.exceptions.InvalidParameterException as e:
            print(f"InvalidParameterException: {e}")
            continue
    return None, 0

# 웹캠 설정
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 현재 시간 기록
    current_time = time.time()
    
    # 이전 Rekognition 요청 후 1초가 지났는지 확인
    if current_time - last_rekognition_request_time >= REQUEST_INTERVAL:
        # 웹캠에서 캡처된 프레임을 Rekognition에서 사용할 수 있도록 변환
        _, buffer = cv2.imencode('.jpg', frame)
        webcam_image = {"Bytes": buffer.tobytes()}

        # Rekognition 호출 및 요청 시간 업데이트
        last_rekognition_request_time = current_time

        # 기존 인물들과 비교하여 ID 할당
        face_id, similarity = compare_face_with_known_faces(webcam_image)

        if face_id:
            # 기존 인물일 경우
            print(f"Matched with ID: {face_id}, Similarity: {similarity:.2f}%")
            cv2.putText(frame, f'ID {face_id} (Similarity: {similarity:.2f}%)', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        else:
            # 새로운 인물일 경우 새로운 ID 할당
            face_id = assign_new_face_id(webcam_image)
            print(f"New face detected, assigned ID: {face_id}")
            cv2.putText(frame, f'New ID {face_id}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

    # 프레임 출력
    cv2.imshow('Re-Identification with AWS Rekognition', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 및 해제
cap.release()
cv2.destroyAllWindows()


### 기존 - 프레임 갱신 주기 추가 + 최근 ID값 유지

In [ ]:
import cv2
import boto3
import time
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# AWS Rekognition 클라이언트 설정
rekognition_client = boto3.client('rekognition', region_name='ap-northeast-2')

# 얼굴 및 ID 정보를 저장할 딕셔너리 (임베딩과 고유 ID)
known_faces = {}  # {face_id: {'image': reference_image, 'name': 'Person X'}}
face_id_counter = 1  # ID 카운터 초기화

# 최근 Rekognition 요청 시간 기록
last_rekognition_request_time = 0
REQUEST_INTERVAL = 1  # 요청 간격 (초)

# 마지막으로 인식한 얼굴 ID 및 유사도 저장 (캐시)
last_face_id = None
last_similarity = None

# 새로운 얼굴이 등장했을 때 ID를 부여하는 함수
def assign_new_face_id(reference_image):
    global face_id_counter
    face_id = face_id_counter
    known_faces[face_id] = {'image': reference_image, 'name': f'Person {face_id}'}
    face_id_counter += 1
    return face_id

# 얼굴 비교 함수 (AWS Rekognition)
def compare_face_with_known_faces(target_image):
    for face_id, face_data in known_faces.items():
        try:
            response = rekognition_client.compare_faces(
                SourceImage=face_data['image'],
                TargetImage=target_image,
                SimilarityThreshold=80  # 유사도 기준값 설정
            )
            if response['FaceMatches']:
                similarity = response['FaceMatches'][0]['Similarity']
                return face_id, similarity
        except rekognition_client.exceptions.InvalidParameterException as e:
            print(f"InvalidParameterException: {e}")
            continue
    return None, 0

# 웹캠 설정
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 현재 시간 기록
    current_time = time.time()
    
    # 이전 Rekognition 요청 후 1초가 지났는지 확인
    if current_time - last_rekognition_request_time >= REQUEST_INTERVAL:
        # 웹캠에서 캡처된 프레임을 Rekognition에서 사용할 수 있도록 변환
        _, buffer = cv2.imencode('.jpg', frame)
        webcam_image = {"Bytes": buffer.tobytes()}

        # Rekognition 호출 및 요청 시간 업데이트
        last_rekognition_request_time = current_time

        # 기존 인물들과 비교하여 ID 할당
        face_id, similarity = compare_face_with_known_faces(webcam_image)

        if face_id:
            # 기존 인물일 경우
            print(f"Matched with ID: {face_id}, Similarity: {similarity:.2f}%")
            last_face_id = face_id  # 마지막 인식된 얼굴 ID 저장
            last_similarity = similarity  # 마지막 유사도 저장
        else:
            # 새로운 인물일 경우 새로운 ID 할당
            face_id = assign_new_face_id(webcam_image)
            print(f"New face detected, assigned ID: {face_id}")
            last_face_id = face_id  # 새로운 얼굴 ID 저장
            last_similarity = 100  # 새 얼굴이므로 100% 유사도 저장

    # 마지막으로 인식된 얼굴 ID 및 유사도가 있는 경우 화면에 출력
    if last_face_id is not None:
        cv2.putText(frame, f'ID {last_face_id} (Similarity: {last_similarity:.2f}%)', (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # 프레임 출력
    cv2.imshow('Re-Identification with AWS Rekognition', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 및 해제
cap.release()
cv2.destroyAllWindows()


### 기존 - 프레임 갱신 주기 추가 + 최근 ID값 유지 + 다중 객체 인식(detect_faces)

In [ ]:
import cv2
import boto3
import time
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# AWS Rekognition 클라이언트 설정
rekognition_client = boto3.client('rekognition', region_name='ap-northeast-2')

# 얼굴 및 ID 정보를 저장할 딕셔너리 (임베딩과 고유 ID)
known_faces = {}  # {face_id: {'image': reference_image, 'name': 'Person X'}}
face_id_counter = 1  # ID 카운터 초기화

# 최근 Rekognition 요청 시간 기록
last_rekognition_request_time = 0
REQUEST_INTERVAL = 1  # 요청 간격 (초)

# 마지막으로 인식한 얼굴 정보를 저장하는 리스트 (캐시)
last_face_results = []

# 새로운 얼굴이 등장했을 때 ID를 부여하는 함수
def assign_new_face_id(reference_image):
    global face_id_counter
    face_id = face_id_counter
    known_faces[face_id] = {'image': reference_image, 'name': f'Person {face_id}'}
    face_id_counter += 1
    return face_id

# 얼굴 비교 함수 (AWS Rekognition)
def compare_face_with_known_faces(target_image):
    for face_id, face_data in known_faces.items():
        try:
            response = rekognition_client.compare_faces(
                SourceImage=face_data['image'],
                TargetImage=target_image,
                SimilarityThreshold=80  # 유사도 기준값 설정
            )
            if response['FaceMatches']:
                similarity = response['FaceMatches'][0]['Similarity']
                return face_id, similarity
        except rekognition_client.exceptions.InvalidParameterException as e:
            print(f"InvalidParameterException: {e}")
            continue
    return None, 0

# 얼굴 탐지 함수 (AWS Rekognition)
def detect_faces_in_frame(frame):
    _, buffer = cv2.imencode('.jpg', frame)
    image_bytes = buffer.tobytes()

    # AWS Rekognition의 DetectFaces 사용
    try:
        response = rekognition_client.detect_faces(
            Image={"Bytes": image_bytes},
            Attributes=['ALL']  # 모든 속성 탐지
        )
        return response.get('FaceDetails', [])
    except Exception as e:
        print(f"Error detecting faces: {e}")
        return []

# 웹캠 설정
cap = cv2.VideoCapture(2)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 현재 시간 기록
    current_time = time.time()
    
    # 이전 Rekognition 요청 후 1초가 지났는지 확인
    if current_time - last_rekognition_request_time >= REQUEST_INTERVAL:
        # 얼굴 탐지 (프레임 내 여러 얼굴 탐지)
        detected_faces = detect_faces_in_frame(frame)

        # Rekognition 호출 및 요청 시간 업데이트
        last_rekognition_request_time = current_time

        # 여러 얼굴에 대해 Rekognition 처리
        last_face_results = []
        for face_detail in detected_faces:
            # 얼굴 위치 정보 (BoundingBox)
            box = face_detail['BoundingBox']
            left = int(box['Left'] * frame.shape[1])
            top = int(box['Top'] * frame.shape[0])
            width = int(box['Width'] * frame.shape[1])
            height = int(box['Height'] * frame.shape[0])

            # 얼굴 영역 추출
            face_image = frame[top:top+height, left:left+width]
            _, face_buffer = cv2.imencode('.jpg', face_image)
            face_bytes = {"Bytes": face_buffer.tobytes()}

            # 기존 인물들과 비교하여 ID 할당
            face_id, similarity = compare_face_with_known_faces(face_bytes)

            if face_id:
                # 기존 인물일 경우
                print(f"Matched with ID: {face_id}, Similarity: {similarity:.2f}%")
            else:
                # 새로운 인물일 경우 새로운 ID 할당
                face_id = assign_new_face_id(face_bytes)
                print(f"New face detected, assigned ID: {face_id}")
                similarity = 100

            # 얼굴 정보 (ID, 유사도, 위치)를 저장
            last_face_results.append((face_id, similarity, (left, top, width, height)))

    # 마지막으로 인식된 얼굴 정보가 있을 경우 출력
    for face_id, similarity, (left, top, width, height) in last_face_results:
        # 얼굴 ID 및 유사도 출력
        cv2.putText(frame, f'ID {face_id} (Similarity: {similarity:.2f}%)', (left, top - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        # 얼굴 영역에 사각형 그리기
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)

    # 프레임 출력
    cv2.imshow('Re-Identification with AWS Rekognition', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 및 해제
cap.release()
cv2.destroyAllWindows()


### 기존 - 프레임 갱신 주기 추가 + 최근 ID값 유지 + 다중 객체 인식(detect_faces) + 유효한 이미지 확인

In [ ]:
import cv2
import boto3
import time
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# AWS Rekognition 클라이언트 설정
rekognition_client = boto3.client('rekognition', region_name='ap-northeast-2')

# 얼굴 및 ID 정보를 저장할 딕셔너리 (임베딩과 고유 ID)
known_faces = {}  # {face_id: {'image': reference_image, 'name': 'Person X'}}
face_id_counter = 1  # ID 카운터 초기화

# 최근 Rekognition 요청 시간 기록
last_rekognition_request_time = 0
REQUEST_INTERVAL = 1  # 요청 간격 (초)

# 마지막으로 인식한 얼굴 정보를 저장하는 리스트 (캐시)
last_face_results = []

# 새로운 얼굴이 등장했을 때 ID를 부여하는 함수
def assign_new_face_id(reference_image):
    global face_id_counter
    face_id = face_id_counter
    known_faces[face_id] = {'image': reference_image, 'name': f'Person {face_id}'}
    face_id_counter += 1
    return face_id

# 얼굴 비교 함수 (AWS Rekognition)
def compare_face_with_known_faces(target_image):
    for face_id, face_data in known_faces.items():
        try:
            response = rekognition_client.compare_faces(
                SourceImage=face_data['image'],
                TargetImage=target_image,
                SimilarityThreshold=80  # 유사도 기준값 설정
            )
            if response['FaceMatches']:
                similarity = response['FaceMatches'][0]['Similarity']
                return face_id, similarity
        except rekognition_client.exceptions.InvalidParameterException as e:
            print(f"InvalidParameterException: {e}")
            continue
    return None, 0

# 얼굴 탐지 함수 (AWS Rekognition)
def detect_faces_in_frame(frame):
    _, buffer = cv2.imencode('.jpg', frame)
    image_bytes = buffer.tobytes()

    # AWS Rekognition의 DetectFaces 사용
    try:
        response = rekognition_client.detect_faces(
            Image={"Bytes": image_bytes},
            Attributes=['ALL']  # 모든 속성 탐지
        )
        return response.get('FaceDetails', [])
    except Exception as e:
        print(f"Error detecting faces: {e}")
        return []

# 웹캠 설정
cap = cv2.VideoCapture(2)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 현재 시간 기록
    current_time = time.time()
    
    # 이전 Rekognition 요청 후 1초가 지났는지 확인
    if current_time - last_rekognition_request_time >= REQUEST_INTERVAL:
        # 얼굴 탐지 (프레임 내 여러 얼굴 탐지)
        detected_faces = detect_faces_in_frame(frame)

        # Rekognition 호출 및 요청 시간 업데이트
        last_rekognition_request_time = current_time

        # 여러 얼굴에 대해 Rekognition 처리
        last_face_results = []
        for face_detail in detected_faces:
            # 얼굴 위치 정보 (BoundingBox)
            box = face_detail['BoundingBox']
            left = int(box['Left'] * frame.shape[1])
            top = int(box['Top'] * frame.shape[0])
            width = int(box['Width'] * frame.shape[1])
            height = int(box['Height'] * frame.shape[0])

            # 얼굴 영역 추출
            face_image = frame[top:top+height, left:left+width]

            # 얼굴 영역이 비어있는지 확인
            if face_image is None or face_image.size == 0:
                print("Empty face image, skipping...")
                continue

            # 얼굴 이미지를 인코딩하여 Rekognition에 사용할 수 있도록 변환
            _, face_buffer = cv2.imencode('.jpg', face_image)
            face_bytes = {"Bytes": face_buffer.tobytes()}

            # 기존 인물들과 비교하여 ID 할당
            face_id, similarity = compare_face_with_known_faces(face_bytes)

            if face_id:
                # 기존 인물일 경우
                print(f"Matched with ID: {face_id}, Similarity: {similarity:.2f}%")
            else:
                # 새로운 인물일 경우 새로운 ID 할당
                face_id = assign_new_face_id(face_bytes)
                print(f"New face detected, assigned ID: {face_id}")
                similarity = 100

            # 얼굴 정보 (ID, 유사도, 위치)를 저장
            last_face_results.append((face_id, similarity, (left, top, width, height)))

    # 마지막으로 인식된 얼굴 정보가 있을 경우 출력
    for face_id, similarity, (left, top, width, height) in last_face_results:
        # 얼굴 ID 및 유사도 출력
        cv2.putText(frame, f'ID {face_id}', (left, top - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        # 얼굴 영역에 사각형 그리기
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)

    # 프레임 출력
    cv2.imshow('Re-Identification with AWS Rekognition', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 및 해제
cap.release()
cv2.destroyAllWindows()


### 바운딩 박스 크기로 객체 검출 시도 차단

In [ ]:
import cv2
import boto3
import time
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# AWS Rekognition 클라이언트 설정
rekognition_client = boto3.client('rekognition', region_name='ap-northeast-2')

# 얼굴 및 ID 정보를 저장할 딕셔너리 (임베딩과 고유 ID)
known_faces = {}  # {face_id: {'image': reference_image, 'name': 'Person X'}}
face_id_counter = 1  # ID 카운터 초기화

# 최근 Rekognition 요청 시간 기록
last_rekognition_request_time = 0
REQUEST_INTERVAL = 1  # 요청 간격 (초)

# 바운딩박스의 크기 기준 임계값 설정 (프레임 크기의 1% 이하인 경우 필터링)
MIN_BOX_SIZE_RATIO = 0.008  # 바운딩박스가 전체 프레임의 0.8% 이하일 경우 검출 무시

# 새로운 얼굴이 등장했을 때 ID를 부여하는 함수
def assign_new_face_id(reference_image):
    global face_id_counter
    face_id = face_id_counter
    known_faces[face_id] = {'image': reference_image, 'name': f'Person {face_id}'}
    face_id_counter += 1
    return face_id

# 얼굴 비교 함수 (AWS Rekognition)
def compare_face_with_known_faces(target_image):
    for face_id, face_data in known_faces.items():
        try:
            response = rekognition_client.compare_faces(
                SourceImage=face_data['image'],
                TargetImage=target_image,
                SimilarityThreshold=80  # 유사도 기준값 설정
            )
            if response['FaceMatches']:
                similarity = response['FaceMatches'][0]['Similarity']
                return face_id, similarity
        except rekognition_client.exceptions.InvalidParameterException as e:
            print(f"InvalidParameterException: {e}")
            continue
    return None, 0

# 얼굴 탐지 함수 (AWS Rekognition)
def detect_faces_in_frame(frame):
    _, buffer = cv2.imencode('.jpg', frame)
    image_bytes = buffer.tobytes()

    # AWS Rekognition의 DetectFaces 사용
    try:
        response = rekognition_client.detect_faces(
            Image={"Bytes": image_bytes},
            Attributes=['ALL']  # 모든 속성 탐지
        )
        return response.get('FaceDetails', [])
    except Exception as e:
        print(f"Error detecting faces: {e}")
        return []

# 웹캠 설정
cap = cv2.VideoCapture(2)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 현재 시간 기록
    current_time = time.time()
    
    # 이전 Rekognition 요청 후 1초가 지났는지 확인
    if current_time - last_rekognition_request_time >= REQUEST_INTERVAL:
        # 얼굴 탐지 (프레임 내 여러 얼굴 탐지)
        detected_faces = detect_faces_in_frame(frame)

        # Rekognition 호출 및 요청 시간 업데이트
        last_rekognition_request_time = current_time

        # 프레임 크기 구하기
        frame_height, frame_width = frame.shape[:2]

        # 여러 얼굴에 대해 Rekognition 처리
        last_face_results = []
        for face_detail in detected_faces:
            # 얼굴 위치 정보 (BoundingBox)
            box = face_detail['BoundingBox']
            left = int(box['Left'] * frame.shape[1])
            top = int(box['Top'] * frame.shape[0])
            width = int(box['Width'] * frame.shape[1])
            height = int(box['Height'] * frame.shape[0])

            # 바운딩박스 크기 비율 계산 (프레임에 대한 비율)
            box_size_ratio = (width * height) / (frame_width * frame_height)
            
            # 바운딩박스가 임계값보다 작으면 건너뛰기
            if box_size_ratio < MIN_BOX_SIZE_RATIO:
                print(f"Bounding box too small (size ratio: {box_size_ratio:.4f}), skipping...")
                continue

            # 얼굴 영역 추출
            face_image = frame[top:top+height, left:left+width]

            # 얼굴 영역이 비어있는지 확인
            if face_image is None or face_image.size == 0:
                print("Empty face image, skipping...")
                continue

            # 얼굴 이미지를 인코딩하여 Rekognition에 사용할 수 있도록 변환
            _, face_buffer = cv2.imencode('.jpg', face_image)
            face_bytes = {"Bytes": face_buffer.tobytes()}

            # 기존 인물들과 비교하여 ID 할당
            face_id, similarity = compare_face_with_known_faces(face_bytes)

            if face_id:
                # 기존 인물일 경우
                print(f"Matched with ID: {face_id}, Similarity: {similarity:.2f}, AgeRange:  {age_low}-{age_high}")
            else:
                # 새로운 인물일 경우 새로운 ID 할당
                face_id = assign_new_face_id(face_bytes)
                print(f"New face detected, assigned ID: {face_id}")
                similarity = 100

            # AgeRange 정보 추출
            age_range = face_detail.get('AgeRange', {})
            age_low = age_range.get('Low', 'Unknown')
            age_high = age_range.get('High', 'Unknown')

            # 얼굴 정보 (ID, 유사도, 나이 범위, 위치)를 저장
            last_face_results.append((face_id, similarity, (age_low, age_high), (left, top, width, height)))

    # 마지막으로 인식된 얼굴 정보가 있을 경우 출력
    for face_id, similarity, (age_low, age_high), (left, top, width, height) in last_face_results:
        # 얼굴 ID, 유사도 및 나이 범위 출력
        cv2.putText(frame, f'ID {face_id}, Age {age_low}-{age_high}', 
                    (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36, 255, 12), 2)
        # 얼굴 영역에 사각형 그리기
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 1)

    # 프레임 출력
    cv2.imshow('Re-Identification with AWS Rekognition', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 및 해제
cap.release()
cv2.destroyAllWindows()


### 바운딩 박스 크기 + 캐시 사용 + 이미지 유효성 검사 추가

In [ ]:
import cv2
import boto3
import time
import numpy as np

# AWS Rekognition 클라이언트 설정
rekognition_client = boto3.client('rekognition', region_name='ap-northeast-2')

# 얼굴 및 ID 정보를 저장할 딕셔너리 (임베딩과 고유 ID)
known_faces = {}  # {face_id: {'image': reference_image, 'name': 'Person X'}}
face_id_counter = 1  # ID 카운터 초기화

# 최근 Rekognition 요청 시간 기록
last_rekognition_request_time = 0
REQUEST_INTERVAL = 1  # 요청 간격 (초)

# 바운딩박스의 크기 기준 임계값 설정 (프레임 크기의 1% 이하인 경우 필터링)
MIN_BOX_SIZE_RATIO = 0.008  # 바운딩박스가 전체 프레임의 1% 이하일 경우 검출 무시

# API 요청 캐시 (같은 얼굴에 대해 연속 API 요청을 방지)
api_request_cache = {}  # {face_id: last_request_time}
CACHE_EXPIRATION_TIME = 3  # 3초 동안 동일 얼굴에 대해 API 요청 금지

# 마지막으로 인식한 얼굴 정보를 저장하는 리스트 (캐시)
last_face_results = []

# 새로운 얼굴이 등장했을 때 ID를 부여하는 함수
def assign_new_face_id(reference_image):
    global face_id_counter
    face_id = face_id_counter
    known_faces[face_id] = {'image': reference_image, 'name': f'Person {face_id}'}
    face_id_counter += 1
    return face_id

# 얼굴 비교 함수 (AWS Rekognition) - 캐시 확인 및 유효성 검사 추가
def compare_face_with_known_faces(target_image):
    current_time = time.time()
    
    for face_id, face_data in known_faces.items():
        # API 요청 캐시 확인 (동일한 얼굴에 대해 일정 시간 이내 중복 요청 방지)
        if face_id in api_request_cache and current_time - api_request_cache[face_id] < CACHE_EXPIRATION_TIME:
            print(f"Skipping API request for face ID {face_id} (within cache time)")
            continue

        # 얼굴 이미지 유효성 검사
        if 'Bytes' not in target_image or len(target_image['Bytes']) < 1024:
            print("Invalid image for comparison, skipping...")
            continue

        try:
            response = rekognition_client.compare_faces(
                SourceImage=face_data['image'],
                TargetImage=target_image,
                SimilarityThreshold=80  # 유사도 기준값 설정
            )
            if response['FaceMatches']:
                similarity = response['FaceMatches'][0]['Similarity']
                api_request_cache[face_id] = current_time  # 캐시 업데이트
                return face_id, similarity
        except rekognition_client.exceptions.InvalidParameterException as e:
            print(f"InvalidParameterException: {e}")
            continue
    return None, 0

# 얼굴 탐지 함수 (AWS Rekognition)
def detect_faces_in_frame(frame):
    _, buffer = cv2.imencode('.jpg', frame)
    image_bytes = buffer.tobytes()

    # AWS Rekognition의 DetectFaces 사용
    try:
        response = rekognition_client.detect_faces(
            Image={"Bytes": image_bytes},
            Attributes=['ALL']  # 모든 속성 탐지
        )
        return response.get('FaceDetails', [])
    except Exception as e:
        print(f"Error detecting faces: {e}")
        return []

# 웹캠 설정
cap = cv2.VideoCapture(2)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # 현재 시간 기록
    current_time = time.time()
    
    # 이전 Rekognition 요청 후 1초가 지났는지 확인
    if current_time - last_rekognition_request_time >= REQUEST_INTERVAL:
        # 얼굴 탐지 (프레임 내 여러 얼굴 탐지)
        detected_faces = detect_faces_in_frame(frame)

        # Rekognition 호출 및 요청 시간 업데이트
        last_rekognition_request_time = current_time

        # 프레임 크기 구하기
        frame_height, frame_width = frame.shape[:2]

        # 여러 얼굴에 대해 Rekognition 처리
        last_face_results = []
        for face_detail in detected_faces:
            # 얼굴 위치 정보 (BoundingBox)
            box = face_detail['BoundingBox']
            left = int(box['Left'] * frame.shape[1])
            top = int(box['Top'] * frame.shape[0])
            width = int(box['Width'] * frame.shape[1])
            height = int(box['Height'] * frame.shape[0])

            # 바운딩박스 크기 비율 계산 (프레임에 대한 비율)
            box_size_ratio = (width * height) / (frame_width * frame_height)
            
            # 바운딩박스가 임계값보다 작으면 건너뛰기
            if box_size_ratio < MIN_BOX_SIZE_RATIO:
                print(f"Bounding box too small (size ratio: {box_size_ratio:.4f}), skipping...")
                continue

            # 얼굴 영역 추출
            face_image = frame[top:top+height, left:left+width]

            # 얼굴 영역이 비어있는지 확인
            if face_image is None or face_image.size == 0:
                print("Empty face image, skipping...")
                continue

            # 얼굴 이미지를 인코딩하여 Rekognition에 사용할 수 있도록 변환
            _, face_buffer = cv2.imencode('.jpg', face_image)
            face_bytes = {"Bytes": face_buffer.tobytes()}

            # 기존 인물들과 비교하여 ID 할당
            face_id, similarity = compare_face_with_known_faces(face_bytes)

            if face_id:
                # 기존 인물일 경우
                print(f"Matched with ID: {face_id}, Similarity: {similarity:.2f}%")
            else:
                # 새로운 인물일 경우 새로운 ID 할당
                face_id = assign_new_face_id(face_bytes)
                print(f"New face detected, assigned ID: {face_id}")
                similarity = 100

            # 얼굴 정보 (ID, 유사도, 위치)를 저장
            last_face_results.append((face_id, similarity, (left, top, width, height)))

    # 마지막으로 인식된 얼굴 정보가 있을 경우 출력
    for face_id, similarity, (left, top, width, height) in last_face_results:
        # 얼굴 ID 및 유사도 출력
        cv2.putText(frame, f'ID {face_id}', (left, top - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        # 얼굴 영역에 사각형 그리기
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)

    # 프레임 출력
    cv2.imshow('Re-Identification with AWS Rekognition', frame)
    
    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 종료 및 해제
cap.release()
cv2.destroyAllWindows()


### pkl 저장 목록 확인

In [ ]:
import pickle

# 저장된 얼굴 데이터를 확인하는 코드
with open("known_faces.pkl", 'rb') as f:
    known_faces = pickle.load(f)

print(f"Loaded {len(known_faces)} known faces.")
for face_id, data in known_faces.items():
    print(f"ID: {face_id}, Name: {data['name']}")


### pkl 에 빈데이터를 다시 저장

In [ ]:
import pickle

# 빈 얼굴 데이터 초기화
known_faces = {}

# 빈 데이터를 다시 저장
with open("known_faces.pkl", 'wb') as f:
    pickle.dump(known_faces, f)

print("Stored face data has been reset (known_faces.pkl file initialized).")


### 프레임 회복성 개선

In [1]:
import cv2
import boto3
import time
from collections import deque
import numpy as np

# AWS Rekognition 클라이언트 설정
rekognition_client = boto3.client('rekognition', region_name='ap-northeast-2')

# 얼굴 및 ID 정보를 저장할 딕셔너리
known_faces = {}
face_id_counter = 1

# 최근 Rekognition 요청 시간 기록
last_rekognition_request_time = 0
REQUEST_INTERVAL = 1  # 요청 간격 (초)

# 바운딩박스의 크기 기준 임계값 설정
MIN_BOX_SIZE_RATIO = 0.008

# 로컬 캐싱을 위한 deque (최근 5개 프레임의 결과를 저장)
face_cache = deque(maxlen=5)

# 새로운 얼굴 ID 부여 함수
def assign_new_face_id(reference_image):
    global face_id_counter
    face_id = face_id_counter
    known_faces[face_id] = {'image': reference_image, 'name': f'Person {face_id}'}
    face_id_counter += 1
    return face_id

# 얼굴 비교 함수 (AWS Rekognition)
def compare_face_with_known_faces(target_image):
    for face_id, face_data in known_faces.items():
        try:
            response = rekognition_client.compare_faces(
                SourceImage=face_data['image'],
                TargetImage=target_image,
                SimilarityThreshold=80
            )
            if response['FaceMatches']:
                similarity = response['FaceMatches'][0]['Similarity']
                return face_id, similarity
        except rekognition_client.exceptions.InvalidParameterException:
            continue
    return None, 0

# 얼굴 탐지 함수 (AWS Rekognition)
def detect_faces_in_frame(frame):
    _, buffer = cv2.imencode('.jpg', frame)
    image_bytes = buffer.tobytes()
    try:
        response = rekognition_client.detect_faces(
            Image={"Bytes": image_bytes},
            Attributes=['ALL']
        )
        return response.get('FaceDetails', [])
    except Exception as e:
        print(f"Error detecting faces: {e}")
        return []

# 웹캠 설정
cap = cv2.VideoCapture(2)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    frame_height, frame_width = frame.shape[:2]

    # 이전 Rekognition 요청 후 일정 시간이 지났는지 확인
    if current_time - last_rekognition_request_time >= REQUEST_INTERVAL:
        detected_faces = detect_faces_in_frame(frame)
        last_rekognition_request_time = current_time

        face_results = []
        for face_detail in detected_faces:
            box = face_detail['BoundingBox']
            left = int(box['Left'] * frame_width)
            top = int(box['Top'] * frame_height)
            width = int(box['Width'] * frame_width)
            height = int(box['Height'] * frame_height)

            box_size_ratio = (width * height) / (frame_width * frame_height)
            if box_size_ratio < MIN_BOX_SIZE_RATIO:
                continue

            face_image = frame[top:top+height, left:left+width]
            if face_image.size == 0:
                continue

            _, face_buffer = cv2.imencode('.jpg', face_image)
            face_bytes = {"Bytes": face_buffer.tobytes()}

            face_id, similarity = compare_face_with_known_faces(face_bytes)
            if not face_id:
                face_id = assign_new_face_id(face_bytes)
                similarity = 100

            face_results.append((face_id, similarity, (left, top, width, height)))

        face_cache.append(face_results)
    else:
        # 캐시된 결과 사용
        face_results = face_cache[-1] if face_cache else []

    # 얼굴 정보 출력
    for face_id, similarity, (left, top, width, height) in face_results:
        cv2.putText(frame, f'ID {face_id}', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
        cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2)

    cv2.imshow('Re-Identification with AWS Rekognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()